<a href="https://colab.research.google.com/github/MaJu502/lab5_paralela/blob/main/lab5_paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio no. 5 Programación Paralela
## author: Marco Jurado 20308

In [13]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [14]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-p662y5w8
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-p662y5w8
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done


In [15]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [16]:
!pip install pycuda

In [17]:
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


Ahora que el ambiente esta listo para ser ejecutado con CUDA procedemos a realizar los ejercicios de esta hoja de trabajo.

In [18]:
%%cuda --name lab5.cu

#include <stdio.h>
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>


__global__ void vectorAdd(float *a, float *b, float *c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
        printf("Thread no. %d: %.2f + %.2f = %.2f\n", idx, a[idx], b[idx], c[idx]);
    }
}

int main() {
    int n = 500;  // Tamaño de los vectores
    float *h_a, *h_b, *h_c; // Vectores en la CPU
    float *d_a, *d_b, *d_c; // Vectores en la GPU
    size_t size = n * sizeof(float);

    // Alojar memoria en la CPU
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c = (float *)malloc(size);

    if (h_a == nullptr || h_b == nullptr || h_c == nullptr) {
        std::cerr << "Error al alojar memoria en la CPU." << std::endl;
        return 1;
    }

    // Inicializar los vectores en la CPU con valores aleatorios
    srand(time(NULL));
    for (int i = 0; i < n; i++) {
        h_a[i] = static_cast<float>(rand()) / RAND_MAX;
        h_b[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Alojar memoria en la GPU
    cudaError_t cudaStatus;
    cudaStatus = cudaMalloc(&d_a, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_b, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_c, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar datos desde la CPU a la GPU
    cudaStatus = cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Configuración de la cuadrícula y bloque
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Lanzar el kernel de CUDA
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    cudaStatus = cudaGetLastError();
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al lanzar el kernel de CUDA: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar el resultado de la GPU a la CPU
    cudaStatus = cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la GPU a la CPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Imprimir el vector resultante y la suma
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        std::cout << "Resultado[" << i << "]: " << h_c[i] << std::endl;
        sum += h_c[i];
    }
    std::cout << "Suma total: " << sum << std::endl;

    // Liberar memoria
    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

'File written in /content/src/lab5.cu'

In [19]:
!nvcc -arch=sm_75 /content/src/lab5.cu -o "/content/src/lab5.o"

In [20]:
!chmod 755 /content/src/lab5.o
!/content/src/lab5.o

Thread no. 480: 0.16 + 0.58 = 0.73
Thread no. 481: 0.13 + 0.64 = 0.77
Thread no. 482: 0.41 + 0.64 = 1.05
Thread no. 483: 0.60 + 0.59 = 1.19
Thread no. 484: 0.58 + 0.79 = 1.38
Thread no. 485: 0.13 + 0.80 = 0.93
Thread no. 486: 0.13 + 0.97 = 1.10
Thread no. 487: 0.74 + 0.75 = 1.50
Thread no. 488: 0.91 + 0.92 = 1.84
Thread no. 489: 0.81 + 0.81 = 1.62
Thread no. 490: 0.01 + 0.65 = 0.66
Thread no. 491: 0.03 + 0.69 = 0.73
Thread no. 492: 0.17 + 0.03 = 0.21
Thread no. 493: 0.47 + 0.19 = 0.66
Thread no. 494: 0.59 + 0.92 = 1.51
Thread no. 495: 0.66 + 0.75 = 1.41
Thread no. 496: 0.50 + 0.79 = 1.29
Thread no. 497: 0.39 + 0.91 = 1.30
Thread no. 498: 0.43 + 0.99 = 1.41
Thread no. 499: 0.50 + 0.01 = 0.51
Thread no. 64: 0.67 + 0.14 = 0.81
Thread no. 65: 0.91 + 0.52 = 1.43
Thread no. 66: 0.49 + 0.18 = 0.67
Thread no. 67: 0.82 + 0.02 = 0.84
Thread no. 68: 0.44 + 0.63 = 1.07
Thread no. 69: 0.78 + 0.75 = 1.53
Thread no. 70: 0.70 + 0.37 = 1.08
Thread no. 71: 0.93 + 0.97 = 1.90
Thread no. 72: 0.30 + 0.87 =

Vamos a realizar el mismo código pero quitando los prints para poder ver el tiempo de ejecución del programa tal y como fue solicitado para los cálculos de speedup.

In [41]:
%%cuda --name lab5_timed.cu

#include <stdio.h>
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>

__global__ void vectorAdd(float *a, float *b, float *c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    int n = 1000000;  // Tamaño de los vectores
    float *h_a, *h_b, *h_c; // Vectores en la CPU
    float *d_a, *d_b, *d_c; // Vectores en la GPU
    size_t size = n * sizeof(float);

    // Alojar memoria en la CPU
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c = (float *)malloc(size);

    if (h_a == nullptr || h_b == nullptr || h_c == nullptr) {
        std::cerr << "Error al alojar memoria en la CPU." << std::endl;
        return 1;
    }

    // Inicializar los vectores en la CPU con valores aleatorios
    srand(time(NULL));
    for (int i = 0; i < n; i++) {
        h_a[i] = static_cast<float>(rand()) / RAND_MAX;
        h_b[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Alojar memoria en la GPU
    cudaError_t cudaStatus;
    cudaStatus = cudaMalloc(&d_a, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_b, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_c, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar datos desde la CPU a la GPU
    cudaStatus = cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Configuración de la cuadrícula y bloque
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Lanzar el kernel de CUDA
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    cudaStatus = cudaGetLastError();
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al lanzar el kernel de CUDA: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar el resultado de la GPU a la CPU
    cudaStatus = cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la GPU a la CPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Imprimir el vector resultante y la suma
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += h_c[i];
    }
    std::cout << "Suma total: " << sum << std::endl;

    // Liberar memoria
    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


'File written in /content/src/lab5_timed.cu'

In [42]:
!nvcc -arch=sm_75 /content/src/lab5_timed.cu -o "/content/src/lab5_timed.o"

In [43]:
%%time

!chmod 755 /content/src/lab5_timed.o
!/content/src/lab5_timed.o

Suma total: 999672
CPU times: user 13.9 ms, sys: 71 µs, total: 13.9 ms
Wall time: 412 ms


Ya que sabemos y hemos obtenido los valores para el codigo en secuencial con cuda vamos a diseñar el formato paralelo y correrlo para obtener el tiempo del mismo y hacer los respectivos calculos.

In [44]:
%%writefile secuencial.cpp

#include <iostream>
#include <cstdlib>
#include <ctime>

int main() {
    int n = 1000000;  // Tamaño de los vectores
    float *h_a, *h_b, *h_c; // Vectores en la CPU
    size_t size = n * sizeof(float);

    // Alojar memoria en la CPU
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c = (float *)malloc(size);

    // Inicializar los vectores en la CPU con valores aleatorios
    srand(time(NULL));
    for (int i = 0; i < n; i++) {
        h_a[i] = static_cast<float>(rand()) / RAND_MAX;
        h_b[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Realizar la suma de los vectores en forma secuencial
    for (int i = 0; i < n; i++) {
        h_c[i] = h_a[i] + h_b[i];
    }

    // Imprimir la suma total
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += h_c[i];
    }
    std::cout << "Suma total: " << sum << std::endl;

    // Liberar memoria
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Overwriting secuencial.cpp


In [45]:
%%bash

g++ secuencial.cpp -o secuencial
time ./secuencial

Suma total: 999872



real	0m0.047s
user	0m0.042s
sys	0m0.005s


In [46]:
import os

# Obtener la cantidad de núcleos (cores) disponibles
num_cores = os.cpu_count()

print("Número de núcleos (cores) disponibles:", num_cores)


Número de núcleos (cores) disponibles: 2


Luego de que tenemos el tiempo de ejecucion de ambos programas procedemos a realizar los calculos de speedup y efficiency de nuestro programa paralelo. Para ello tomaremos un tamaño de 1,000,000 y realizamos la ejecución donde obtenemos lo siguiente:


*   Tiempo secuencial: 47 ms
*   Tiempo paralelo: 13.9 ms

* Speedup = 47 ms / 13.9 ms ≈ 3.38
* Efficiency = 3.38 / 2 ≈ 1.69

